# RECONHECIMENTO COM CNN

# IMPORTANDO O DATASET

In [ ]:
import tensorflow as tf
from keras import layers
from keras import models
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import cv2
from tensorflow import keras

In [ ]:
# IMPORTANDO DATASET
from google.colab import drive
drive.mount('/content/drive') # importa arquivos do google drive para o disco da máquina virtual

In [ ]:
!mkdir dataset

In [ ]:
%cd dataset
!mkdir chars
#!mkdir nums

In [ ]:
!unzip [CAMINHO IMAGENS ZIPADAS]/.zip -d /content/dataset/chars/

In [ ]:
# DEFINE TAMANHO DA IMAGENS APÓS RESIZE
r = 30
c = 20
color = (0,0,0)

# DEFINE CAMINHOS PARA ARQUIVOS DE TREINAMENTO/TESTE E INFERÊNCIA

dir_chars_train = '[CAMINHO]' # DIRETÓRIO COM AS IMAGENS DE TREINAMENTO
dir_chars_train_lbl = '[CAMINHO]' # DIRETÓRIO COM OS LABELS DAS IMAGENS DE TREINAMENTO
filename_chars_train = '[CAMINHO].txt' # ARQUIVO COM O NOME DO ARQUIVO DE CADA IMAGEM

dir_chars_valid = '[CAMINHO]' # DIRETÓRIO COM AS IMAGENS DE VALIDAÇÃO
dir_chars_valid_lbl = '[CAMINHO]' # DIRETÓRIO COM OS LABELS DAS IMAGENS DE VALIDAÇÃO
filename_chars_valid = '[CAMINHO]/.txt' # ARQUIVO COM O NOME DO ARQUIVO DE CADA IMAGEM

dir_chars_test = '[CAMINHO]' # DIRETÓRIO COM AS IMAGENS DE TESTES
dir_chars_test_lbl = '[CAMINHO]' # DIRETÓRIO COM OS LABELS DAS IMAGENS DE TESTES
filename_chars_test = '[CAMINHO]/.txt'# ARQUIVO COM O NOME DO ARQUIVO DE CADA IMAGEM

num_classes = 26

In [ ]:
# FUNÇÃO QUE REDIMENSIONA IMAGENS DOS CARACTERES E ADICIONA UM PADDING
def redimensiona_img(img, r, c, color):
  old_image_height, old_image_width, channels = img.shape

  if old_image_height > old_image_width:
    new_image_width = old_image_height
    new_image_height = old_image_height
  else:
    new_image_width = old_image_width
    new_image_height = old_image_width

  result = np.full((new_image_height,new_image_width, channels), color, dtype=np.uint8)
  x_center = (new_image_width - old_image_width) // 2
  y_center = (new_image_height - old_image_height) // 2

  #print(x_center)
  #print(y_center)
  #print(img.shape)

  result[y_center:y_center+old_image_height,
       x_center:x_center+old_image_width] = img

  img = cv2.resize(result, ( c, r ))

  return img

In [ ]:
# FUNÇÃO QUE REDIMENSIONA IMAGENS DOS CARACTERES
def redimensiona_img_nopad(img, r, c):

  img = cv2.resize(img, ( c, r ))

  return img

In [ ]:
# FUNÇÃO DE LEITURA DAS IMAGENS E LABELS DOS CARACTERES

def read_imgs_n_lbls(dir_imgs, dir_lbl, filenames_txt):
  temp = open(filenames_txt, 'r').read().splitlines()
  im = [dir_imgs + x for x in temp]

  imgs = []
  lbls = []

  for i in range(0, len(im)):
    img = cv2.imread(im[i])

    img = redimensiona_img(img, r, c, color)
    imgs.append(img)
    
    if(dir_lbl != 'N/A'):
      lbl_name = im[i][:-3] + 'txt'
      lbl_name = lbl_name[-20:]
      f_lbl = open(dir_lbl + lbl_name, 'r')
      lbl = f_lbl.readline()
      f_lbl.close()

      lbl = lbl.split()
      lbl = lbl[0]

      lbls.append(lbl)

  return imgs, lbls

In [ ]:
# LENDO IMAGENS DE TREINAMENTO (LETRAS)
[imgs_train, lbls_train] = read_imgs_n_lbls( dir_chars_train, dir_chars_train_lbl, filename_chars_train )
print("fim imagens treinamento")
[imgs_valid, lbls_valid] = read_imgs_n_lbls( dir_chars_valid, dir_chars_valid_lbl, filename_chars_valid )
print("fim imagens validação")
[imgs_test, lbls_test] = read_imgs_n_lbls( dir_chars_test, dir_chars_test_lbl, filename_chars_test )
print("fim imagens testing")

In [ ]:
# PLOTA TAMANHOS DOS SETS DE IMAGENS
print(len(imgs_train))
print(len(imgs_valid))
print(len(imgs_test))

In [ ]:
# concatena conjunto de treino e validação
for i in range( 0, len(imgs_valid) ):
  imagem = imgs_valid[i]
  imgs_train.append(imagem)

print('Tamanho do dataset train+valid = ' + str(len(imgs_train)))

In [ ]:
# concatena conjunto de treino e validação
for i in range( 0, len(lbls_valid) ):
  label = lbls_valid[i]
  lbls_train.append(label)

print('Tamanho dos labels do dataset train+valid = ' + str(len(lbls_train)))

In [ ]:
# REDIMENSIONAR IMAGENS PARA O INTERVALO [0, 1]
imgs_train = tf.keras.utils.normalize(imgs_train, axis=1)
#imgs_valid = tf.keras.utils.normalize(imgs_valid, axis=1)
imgs_test = tf.keras.utils.normalize(imgs_test, axis=1)

In [ ]:
# CONVERTE VETORES DE CLASS EM MATRIZES DE CLASSES BINÁRIAS
lbls_train = keras.utils.to_categorical(lbls_train, num_classes)
lbls_test = keras.utils.to_categorical(lbls_test, num_classes)

# MONTANDO E TREINANDO A CNN

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), padding="same", activation='relu', input_shape=(r, c, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), padding="same", activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), padding="same", activation='relu'))
model.add(layers.Conv2D(64, (1, 1), padding="same", activation='relu'))
model.add(layers.Conv2D(128, (3, 3), padding="same", activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), padding="same", activation='relu'))
model.add(layers.Conv2D(128, (1, 1), padding="same", activation='relu'))
model.add(layers.Conv2D(256, (3, 3), padding="same", activation='relu'))
model.add(layers.Conv2D(512, (3, 3), padding="same", activation='relu'))
model.add(layers.Conv2D(256, (1, 1), padding="same", activation='relu'))
model.add(layers.Conv2D(512, (3, 3), padding="same", activation='relu'))
model.add(layers.Conv2D(155, (1, 1), padding="same", activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(26,activation="softmax"))
model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [ ]:
batch_size = 128
epochs = 15

In [ ]:
# REALIZA O TREINO, SALVA E ENCERRA EXECUÇÃO
model.fit(imgs_train,  lbls_train, batch_size=batch_size, epochs=epochs, validation_split=0.33)

%cd /content/
model_name = 'NOMEMODELOTREINADO.model'
model.save(model_name)
!cp -r /content/num_rec_v01.model -d /content/drive/MyDrive/Experimentos_CNN/
from google.colab import runtime
runtime.unassign()

# AVALIANDO A CNN

In [ ]:
# AVALIA MODELO NO CONJ DE TESTES
score = model.evaluate(imgs_test, lbls_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
# CARREGA MODELO SALVO

dir_models = 'CAMINHO'
model_name = 'MODELO.model'
model = tf.keras.models.load_model(dir_models + model_name)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", 'wb') as f:
  f.write(tflite_model)

## AVALIANDO MODELO

In [ ]:
# Avaliando Modelos

import tensorflow as tf
import numpy as np
import cv2

In [ ]:
# IMPORTANDO DATASET
from google.colab import drive
drive.mount('/content/drive') # importa arquivos do google drive para o disco da máquina virtual
print("conexão com o google drive estabelecida")
!mkdir dataset
%cd dataset

!unzip [CAMINHO IMAGENS ZIPADAS].zip -d /content/dataset/
print("imagens e labels descarregados")

# DEFINE TAMANHO DA IMAGENS APÓS RESIZE
r = 30
c = 20
color = (0,0,0)

# DEFINE CAMINHOS PARA ARQUIVOS DE TREINAMENTO/TESTE E INFERÊNCIA ------- CHARS ORIGINAIS RECORTADOS

dir_chars_test = '/content/dataset/'
dir_chars_test_lbl = '/content/dataset/'
filename_chars_test = '[CAMINHO PARA ARQUIVO COM NOMES DAS IMAGENS]filename_numbers_avageral.txt'

num_classes = 10

In [ ]:
# FUNÇÃO DE LEITURA DAS IMAGENS E LABELS DOS CARACTERES

def read_imgs_n_lbls(dir_imgs, dir_lbl, filenames_txt):
  temp = open(filenames_txt, 'r').read().splitlines()
  im = [dir_imgs + x for x in temp]

  imgs = []
  lbls = []

  for i in range(0, len(im)):
    img = cv2.imread(im[i])

    img = redimensiona_img(img, r, c, color)
    imgs.append(img)

    if(dir_lbl != 'N/A'):
      lbl_name = im[i][:-3] + 'txt'
      lbl_name = lbl_name[-20:]
      f_lbl = open(dir_lbl + lbl_name, 'r')
      lbl = f_lbl.readline()
      f_lbl.close()

      lbl = lbl.split()
      lbl = lbl[0]

      lbls.append(lbl)

  print("leitura de imagens e rótulos após reajustes finalizada")
  return imgs, lbls

In [ ]:
# RODA O TESTE

# LENDO IMAGENS DE TESTES
[imgs_test, lbls_test] = read_imgs_n_lbls( dir_chars_test, 'N/A', filename_chars_test )
print("fim imagens teste")

# PLOTA TAMANHO DO SET DE IMAGENS
print("Número de imagens do conjunto de testes: " + str( len(imgs_test) ) )
print("Número de labels do conjunto de testes: " + str( len(lbls_test) ) )

# REDIMENSIONAR IMAGENS PARA O INTERVALO [0, 1]
imgs_test = tf.keras.utils.normalize(imgs_test, axis=1)

# CONVERTE VETORES DE CLASS EM MATRIZES DE CLASSES BINÁRIAS
#lbls_test = keras.utils.to_categorical(lbls_test, num_classes)

# CARREGA MODELO SALVO
dir_models = '[CAMINHO DIRETÓRIO COM MODELOS SALVOS]'
model_name = 'MODEL.model'
model = tf.keras.models.load_model(dir_models + model_name)

# AVALIA MODELO NO CONJ DE TESTES
score = model.evaluate(imgs_test, lbls_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
# ANOTA PREDIÇÕES
%cd /content/
with open('RESULTADOS.txt', 'w') as f:
    f.write('Resultados')
f.close()

for i in range( 0, len(imgs_test) ):
  img = imgs_test[i]
  preds = model.predict(np.array([img]))
  classes = np.argmax(preds, axis = 1)
  with open('RESULTADOS.txt', 'a') as f:
    f.write('\n')
    f.writelines(str(classes))
  f.close()

!cp -r /content/RESULTADOS.txt -d /content/drive/MyDrive/